# Test Snowflake RAG Service Implementation

This notebook tests the Snowflake service implementation for our RAG chat application.

In [39]:
import os
import tempfile
from snowflake_service import SnowflakeService

# Set up connection parameters (replace with your actual credentials)
# connection_params = {
#     "account": os.getenv("SNOWFLAKE_ACCOUNT"),
#     "user": os.getenv("SNOWFLAKE_USER"),
#     "password": os.getenv("SNOWFLAKE_PASSWORD"),
#     "role": os.getenv("SNOWFLAKE_ROLE"),
#     "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
#     "database": "RAG_CHAT_DB",
#     "schema": "RAG_CHAT_SCHEMA"
# }
connection_params = {
    "account": "blcquff-dcb49840",
    "user": "TAZ16",
    "password": "Srinijani@95",
    "role": "ACCOUNTADMIN",
    "warehouse": "COMPUTE_WH",
    "database": "RAG_CHAT_DB",
    "schema": "RAG_CHAT_SCHEMA"
}
print("Connection parameters:", connection_params)

Connection parameters: {'account': 'blcquff-dcb49840', 'user': 'TAZ16', 'password': 'Srinijani@95', 'role': 'ACCOUNTADMIN', 'warehouse': 'COMPUTE_WH', 'database': 'RAG_CHAT_DB', 'schema': 'RAG_CHAT_SCHEMA'}


## Initialize Snowflake Service

In [40]:
# Initialize Snowflake service
service = SnowflakeService(connection_params)
print("Service initialized successfully!")

✅ In Groundedness, input source will be set to __record__.main_input or `Select.RecordInput` .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
instrumenting <class 'trulens.core.feedback.feedback.Feedback'> for base <class 'trulens.core.feedback.feedback.Feedback'>
	instrumenting __call__
skipping base <class 'trulens.core.schema.feedback.FeedbackDefinition'> because of class
skipping base <class 'trulens.core.utils.pyschema.WithClassInfo'> because of class
skipping base <class 'trulens.core.utils.serial.SerialModel'> because of class


OperationalError: (sqlite3.OperationalError) no such table: trulens_apps
[SQL: SELECT trulens_apps.app_id AS trulens_apps_app_id, trulens_apps.app_name AS trulens_apps_app_name, trulens_apps.app_version AS trulens_apps_app_version, trulens_apps.app_json AS trulens_apps_app_json 
FROM trulens_apps 
WHERE trulens_apps.app_id = ?
 LIMIT ? OFFSET ?]
[parameters: ('app_hash_2fd14560bf7d87e7add5ab30495d3eec', 1, 0)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

## Test File Upload and Processing

In [28]:
# Create a sample markdown file
test_content = """# Sample Document

This is a test document to verify our RAG implementation.

## Section 1
The capital of France is Paris. It is known as the City of Light.
Paris has many famous landmarks including the Eiffel Tower and the Louvre Museum.

## Section 2
Python is a popular programming language created by Guido van Rossum.
It is known for its simplicity and readability.

## Section 3
Machine learning is a subset of artificial intelligence.
It allows computers to learn from data without being explicitly programmed.
"""

# Create temporary file
with tempfile.NamedTemporaryFile(suffix='.md', delete=False, mode='w') as f:
    f.write(test_content)
    test_file_path = f.name

print(f"Created test file at: {test_file_path}")

# Upload file
with open(test_file_path, 'rb') as f:
    success = service.upload_file(
        file_path=test_file_path,
        file_content=f.read(),
        file_type="MD"
    )

print(f"File upload {'successful' if success else 'failed'}")

Created test file at: /var/folders/_z/ms9rqjt90dq1s5797l_x0ry40000gn/T/tmpgugp2720.md
File upload successful


In [41]:
# Debug command 1 - List files
list_stmt = f"LIST @{service.db_name}.{service.schema_name}.document_stage"
staged_files = service.session.sql(list_stmt).collect()
print("Raw staged files:")
for file in staged_files:
    print(file.asDict())

# Debug command 2 - Try parsing one file
if staged_files:
    file_dict = staged_files[0].asDict()
    print("\nFirst file info:", file_dict)
    staged_path = file_dict.get('name', '')
    clean_path = staged_path.replace('document_stage/', '') if staged_path.startswith('document_stage/') else staged_path
    clean_path = clean_path[:-3] if clean_path.endswith('.gz') else clean_path
    print(f"\nStaged path: {staged_path}")
    print(f"Clean path: {clean_path}")

Raw staged files:
{'name': 'document_stage/FILE_NAME.md', 'size': 31622, 'md5': 'fd38bb5bffb6b0c00980fd8f96cb0689', 'last_modified': 'Sat, 1 Feb 2025 21:48:46 GMT'}

First file info: {'name': 'document_stage/FILE_NAME.md', 'size': 31622, 'md5': 'fd38bb5bffb6b0c00980fd8f96cb0689', 'last_modified': 'Sat, 1 Feb 2025 21:48:46 GMT'}

Staged path: document_stage/FILE_NAME.md
Clean path: FILE_NAME.md


In [29]:
# Debug command 3 - Try just the PARSE_DOCUMENT step for one file
if staged_files:
    file_dict = staged_files[0].asDict()
    clean_path = file_dict['name'].replace('document_stage/', '').replace('.gz', '')
    
    parse_stmt = f"""
    SELECT SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
        @{service.db_name}.{service.schema_name}.document_stage,
        '{clean_path}',
        {{'mode': 'LAYOUT'}}
    );
    """
    print("Parse statement:")
    print(parse_stmt)
    try:
        result = service.session.sql(parse_stmt).collect()
        print("\nParse result:", result)
    except Exception as e:
        print("\nParse error:", str(e))

Parse statement:

    SELECT SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
        @RAG_CHAT_DB.RAG_CHAT_SCHEMA.document_stage,
        'tmp_c49wlyt.md',
        {'mode': 'LAYOUT'}
    );
    

Parse result: [Row(SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
        @RAG_CHAT_DB.RAG_CHAT_SCHEMA.DOCUMENT_STAGE,
        'TMP_C49WLYT.MD',
        {'MODE': 'LAYOUT'}
    )='{\n  "errorInformation": "The provided file format .txt isn\'t supported. Supported formats: [\'.docx\', \'.pptx\', \'.pdf\']"\n}')]


In [30]:
# Verify table exists and structure
show_table = f"DESC TABLE {service.db_name}.{service.schema_name}.DOCUMENT_CHUNKS"
try:
    result = service.session.sql(show_table).collect()
    print("Table structure:")
    for row in result:
        print(row.asDict())
except Exception as e:
    print("Table error:", str(e))

Table structure:
{'name': 'RELATIVE_PATH', 'type': 'VARCHAR(512)', 'kind': 'COLUMN', 'null?': 'Y', 'default': None, 'primary key': 'N', 'unique key': 'N', 'check': None, 'expression': None, 'comment': None, 'policy name': None, 'privacy domain': None}
{'name': 'FILE_NAME', 'type': 'VARCHAR(256)', 'kind': 'COLUMN', 'null?': 'Y', 'default': None, 'primary key': 'N', 'unique key': 'N', 'check': None, 'expression': None, 'comment': None, 'policy name': None, 'privacy domain': None}
{'name': 'CHUNK_ID', 'type': 'NUMBER(38,0)', 'kind': 'COLUMN', 'null?': 'Y', 'default': 'IDENTITY START 1 INCREMENT 1 NOORDER', 'primary key': 'N', 'unique key': 'N', 'check': None, 'expression': None, 'comment': None, 'policy name': None, 'privacy domain': None}
{'name': 'CHUNK_TEXT', 'type': 'VARCHAR(16777216)', 'kind': 'COLUMN', 'null?': 'Y', 'default': None, 'primary key': 'N', 'unique key': 'N', 'check': None, 'expression': None, 'comment': None, 'policy name': None, 'privacy domain': None}
{'name': 'CREATE

In [42]:
# Process staged files
processed_files = service.process_staged_files()
print(f"Processed files: {processed_files}")

Found 1 files in stage
Sample file info: {'name': 'document_stage/FILE_NAME.md', 'size': 31622, 'md5': 'fd38bb5bffb6b0c00980fd8f96cb0689', 'last_modified': 'Sat, 1 Feb 2025 21:48:46 GMT'}
Processing file: FILE_NAME.md (Path: FILE_NAME.md)
File extension: .md
Result when getting markdown: [Row($1='# TickTick MCP Server Implementation Plan'), Row($1=None), Row($1='## Overview'), Row($1="An MCP server implementation that integrates TickTick's task management capabilities with LLM-powered planning and organization features. The server will enable LLMs like Claude to interact with TickTick for creating and managing tasks"), Row($1=None), Row($1='## Core Features'), Row($1=None), Row($1='### 1. Authentication & Setup'), Row($1='- **OAuth2 Integration**'), Row($1='  - Implementation: Complex'), Row($1='  - Priority: High'), Row($1='  - Components:'), Row($1='    - OAuth2 flow implementation'), Row($1='    - Token management and refresh'), Row($1='    - Secure credential storage'), Row($1='  -

## Test Document Retrieval

In [43]:
# Test context retrieval
test_query = "Give the plan for the MCP project?"
context = service.retrieve_context(test_query)

print("Query:", test_query)
print("\nRetrieved Context:")
for chunk in context:
    print(f"\n- {chunk['CHUNK_TEXT']}")

Query: Give the plan for the MCP project?

Retrieved Context:

- # TickTick MCP Server Implementation Plan

## Overview
An MCP server implementation that integrates TickTick's task management capabilities with LLM-powered planning and organization features. The server will enable LLMs like Claude to interact with TickTick for creating and managing tasks

## Core Features

### 1. Authentication & Setup
- **OAuth2 Integration**
  - Implementation: Complex
  - Priority: High
  - Components:
    - OAuth2 flow implementation
    - Token management and refresh
    - Secure credential storage
  - Technical Considerations:
    - Environment variables for client credentials
    - Secure token storage
    - Token refresh handling

- 2. **Update Responsibility:**
   - The MCP server is responsible for keeping resources up-to-date
   - Updates happen through:
     - Regular polling of TickTick API
     - Response to client subscription requests
     - After successful tool executions
     - Backgr

## Test RAG with TruLens Monitoring

In [44]:
# Test complete RAG pipeline
test_queries = [
    "Give the plan for the MCP project?",
]

for query in test_queries:
    
    answer, metadata = service.chat(query)
    print(answer)
        
        
        

Error generating answer: (1409): More than one active session is detected. When you call function 'udf' or use decorator '@udf', you must specify the 'session' parameter if you created multiple sessions.Alternatively, you can use 'session.udf.register' to register UDFs
Error storing chat message: (1304): 01ba1c20-0004-7b12-0004-b9e300018d56: 001003 (42000): SQL compilation error:
syntax error line 6 at position 169 unexpected 's'.
syntax error line 6 at position 217 unexpected 'planning'.
parse error line 6 at position 351 near '110'.
parse error line 6 at position 371 near '110'.
parse error line 6 at position 404 near '110'.
parse error line 6 at position 459 near '110'.
parse error line 6 at position 479 near '110'.
parse error line 6 at position 530 near '110'.
parse error line 6 at position 566 near '110'.
parse error line 6 at position 599 near '110'.
parse error line 6 at position 682 near '110'.
parse error line 6 at position 710 near '110'.
parse error line 6 at position 904 n

## Test Chat History

In [24]:
# Get chat history for the last conversation
chat_id = metadata['chat_id']
history = service.get_chat_history(chat_id)

print(f"Chat History (ID: {chat_id}):")
for msg in history:
    print(f"\n[{msg['ROLE']}] {msg['CONTENT']}")
    if msg['METADATA']:
        print(f"Metadata: {msg['METADATA']}")

KeyError: 'chat_id'

## Cleanup

In [ ]:
# Remove temporary test file
import os
if os.path.exists(test_file_path):
    os.remove(test_file_path)
    print(f"Removed test file: {test_file_path}")